<a href="https://colab.research.google.com/github/hjhj6411/yolomt/blob/main/yolomt_ir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 기존 설치 시도 찌꺼기 제거 (혹시 모를 충돌 방지)
!pip uninstall -y onnx onnxsim protobuf

# 2. Python 3.12와 호환되는 안정적인 버전 설치
# onnx>=1.16.0 : Python 3.12 지원
# onnxsim>=0.4.36 : 최신 ONNX와 호환
!pip install "onnx>=1.16.0" "onnxsim>=0.4.36"

print("✓ ONNX & ONNX-Simplifier installed successfully")

In [ ]:
# 1. PyTorch 2.2.0 + CUDA 11.8 설치
# (Linux/Colab 기준)
!pip install torch==2.2.0+cu118 torchvision==0.17.0+cu118 torchaudio==2.2.0+cu118 --index-url https://download.pytorch.org/whl/cu118

# 2. NumPy 1.26.4 설치 (매우 중요)
!pip install numpy==1.26.4

In [ ]:
print("="*70)
print("YOLOMT Training Environment Check")
print("="*70)

import sys
import torch

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("="*70)

YOLOMT Training Environment Check
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch version: 2.2.0+cu118
CUDA available: True
CUDA version: 11.8
GPU: Tesla T4


In [ ]:
print("="*70)
print("🚀 YOLOMT with 300W-LP Dataset - Complete Pipeline")
print("="*70)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
import os
import glob
from tqdm import tqdm
import onnx
from onnxsim import simplify

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✓ Device: {device}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

🚀 YOLOMT with 300W-LP Dataset - Complete Pipeline


Installing onnxruntime by `/usr/bin/python3 -m pip install onnxruntime`, please wait for a moment..


✓ Device: cuda
✓ GPU: Tesla T4


In [ ]:
!cp "/content/drive/MyDrive/archive.zip" "/content/temp_dataset.zip"

!unzip -qq "/content/temp_dataset.zip" -d "/content/dataset"

print("압축 해제 완료!")

압축 해제 완료!


In [ ]:
DATA_ROOT = '/content/dataset/300W_LP'

USE_DUMMY_DATA = not os.path.exists(DATA_ROOT)

if USE_DUMMY_DATA:
    print("\n⚠️  300W-LP not found, using dummy data for demonstration")
    print("   (성능은 나오지 않지만 전체 파이프라인은 테스트 가능)")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RepConv(nn.Module):
    def __init__(self, c1, c2, k=3, s=1, p=None, g=1, act=True):
        super().__init__()
        self.deploy = False
        self.act = nn.SiLU() if act else nn.Identity()
        self.g = g  # 그룹 정보 저장 (매우 중요)

        if p is None: p = k // 2

        self.conv_kxk = nn.Sequential(
            nn.Conv2d(c1, c2, k, s, p, groups=g, bias=False),
            nn.BatchNorm2d(c2)
        )

        self.conv_1x1 = nn.Sequential(
            nn.Conv2d(c1, c2, 1, s, 0, groups=g, bias=False),
            nn.BatchNorm2d(c2)
        ) if k > 1 else None

        self.bn = nn.BatchNorm2d(c1) if c2 == c1 and s == 1 else None

    def forward(self, x):
        if self.deploy:
            return self.act(self.conv_fused(x))

        y = self.conv_kxk(x)
        if self.conv_1x1:
            y += self.conv_1x1(x)
        if self.bn:
            y += self.bn(x)
        return self.act(y)

    def fuse(self):
        if self.deploy: return
        kernel, bias = self._get_equivalent_kernel_bias()

        # Fused Conv 생성 (groups 설정 유지)
        self.conv_fused = nn.Conv2d(
            self.conv_kxk[0].in_channels,
            self.conv_kxk[0].out_channels,
            self.conv_kxk[0].kernel_size,
            self.conv_kxk[0].stride,
            self.conv_kxk[0].padding,
            groups=self.conv_kxk[0].groups, # groups 값 전달
            bias=True
        )
        self.conv_fused.weight.data = kernel
        self.conv_fused.bias.data = bias

        # 가지치기
        for attr in ['conv_kxk', 'conv_1x1', 'bn']:
            if hasattr(self, attr): delattr(self, attr)
        self.deploy = True

    def _get_equivalent_kernel_bias(self):
        k3, b3 = self._fuse_bn(self.conv_kxk[0], self.conv_kxk[1])
        k1, b1 = 0, 0
        if self.conv_1x1:
            k1, b1 = self._fuse_bn(self.conv_1x1[0], self.conv_1x1[1])
            pad = (self.conv_kxk[0].kernel_size[0] - 1) // 2
            k1 = F.pad(k1, [pad]*4)
        kb, bb = 0, 0
        if self.bn:
            kb, bb = self._fuse_bn_identity(self.bn)
        return k3 + k1 + kb, b3 + b1 + bb

    @staticmethod
    def _fuse_bn(conv, bn):
        std = (bn.running_var + bn.eps).sqrt()
        t = (bn.weight / std).reshape(-1, 1, 1, 1)
        return conv.weight * t, bn.bias - bn.running_mean * bn.weight / std

    def _fuse_bn_identity(self, bn):
        # 🔧 [수정된 부분] 그룹 연산(Depthwise)일 경우 차원을 맞춰주는 로직 추가
        k_size = self.conv_kxk[0].kernel_size[0]
        in_ch = bn.num_features

        # 중요: self.g > 1이면 (C, 1, K, K) 형태여야 함
        if self.g > 1:
             kernel = torch.zeros((in_ch, 1, k_size, k_size), device=bn.weight.device)
             for i in range(in_ch):
                 kernel[i, 0, k_size//2, k_size//2] = 1
        else:
             kernel = torch.zeros((in_ch, in_ch, k_size, k_size), device=bn.weight.device)
             for i in range(in_ch):
                 kernel[i, i, k_size//2, k_size//2] = 1

        std = (bn.running_var + bn.eps).sqrt()
        t = (bn.weight / std).reshape(-1, 1, 1, 1)
        return kernel * t, bn.bias - bn.running_mean * bn.weight / std


# ==========================================
# 2. 나머지 클래스 (RepBottleneck, RepC2f, YOLOMT)
# ==========================================
# 기존 코드와 동일하지만, RepConv가 수정되었으므로 다시 정의해야 적용됩니다.

class RepBottleneck(nn.Module):
    def __init__(self, c1, c2, shortcut=True, g=1, k=(3, 3), e=0.5):
        super().__init__()
        c_ = int(c2 * e)
        self.cv1 = RepConv(c1, c_, k[0], 1)
        self.cv2 = RepConv(c_, c2, k[1], 1, g=c_)
        self.add = shortcut and c1 == c2

    def forward(self, x):
        return x + self.cv2(self.cv1(x)) if self.add else self.cv2(self.cv1(x))

class RepC2f(nn.Module):
    def __init__(self, c1, c2, n=1, shortcut=False, g=1, e=0.5):
        super().__init__()
        self.c = int(c2 * e)
        # 여기서 k=1일 때 RepConv가 알아서 p=0으로 설정하여 에러 해결됨
        self.cv1 = RepConv(c1, 2 * self.c, 1, 1)
        self.cv2 = RepConv((2 + n) * self.c, c2, 1)
        self.m = nn.ModuleList(RepBottleneck(self.c, self.c, shortcut, g, k=(3, 3), e=1.0) for _ in range(n))

    def forward(self, x):
        y = list(self.cv1(x).chunk(2, 1))
        y.extend(m(y[-1]) for m in self.m)
        return self.cv2(torch.cat(y, 1))

class SimplifiedYOLOMT(nn.Module):
    def __init__(self, nc=1, n_keypoints=68):
        super().__init__()
        self.nc = nc
        self.stem = nn.Sequential(RepConv(3, 32, 3, 2), RepConv(32, 64, 3, 2))
        self.stage1 = RepC2f(64, 64, n=1, shortcut=True)
        self.down1 = RepConv(64, 128, 3, 2)
        self.stage2 = RepC2f(128, 128, n=2, shortcut=True)
        self.down2 = RepConv(128, 256, 3, 2)
        self.stage3 = RepC2f(256, 256, n=2, shortcut=True)
        self.det_head = nn.Sequential(RepConv(256, 128, 3, 1), nn.Conv2d(128, 5 + nc, 1))
        self.kpt_head = nn.Sequential(RepConv(256, 128, 3, 1), nn.Conv2d(128, n_keypoints * 3, 1))

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.down1(x)
        x = self.stage2(x)
        x = self.down2(x)
        x = self.stage3(x)
        return self.det_head(x), self.kpt_head(x)

    def fuse(self):
        for m in self.modules():
            if isinstance(m, RepConv): m.fuse()
        return self

print("✓ Model fixed: AutoPad applied to RepConv")


# ==========================================
# 3. YOLOMT Model (기존 코드와 호환성 유지)
# ==========================================

class SimplifiedYOLOMT(nn.Module):
    """
    YOLOMT Architecture with RepC2f Blocks
    입출력 구조는 기존 Training Loop와 호환되도록 유지함.
    """
    def __init__(self, nc=1, n_keypoints=68):
        super().__init__()
        self.nc = nc

        # Stem: Low-level features [cite: 222]
        self.stem = nn.Sequential(
            RepConv(3, 32, 3, 2, 1), # Stride 2
            RepConv(32, 64, 3, 2, 1) # Stride 2
        )

        # Backbone Stage 1 (P3)
        # 기존 Sequential 대신 RepC2f 사용으로 깊은 특징 추출
        self.stage1 = RepC2f(64, 64, n=1, shortcut=True)
        self.down1 = RepConv(64, 128, 3, 2, 1) # Downsample

        # Backbone Stage 2 (P4)
        self.stage2 = RepC2f(128, 128, n=2, shortcut=True)
        self.down2 = RepConv(128, 256, 3, 2, 1) # Downsample

        # Backbone Stage 3 (P5)
        self.stage3 = RepC2f(256, 256, n=2, shortcut=True)

        # Detection Head (Decoupled Head [cite: 284])
        # 기존 Training Loop의 단순함에 맞춰 출력 채널만 조정
        self.det_head = nn.Sequential(
            RepConv(256, 128, 3, 1),
            nn.Conv2d(128, 5 + nc, 1) # Box(4) + Conf(1) + Class(nc)
        )

        # Keypoint Head (Decoupled Head)
        self.kpt_head = nn.Sequential(
            RepConv(256, 128, 3, 1),
            nn.Conv2d(128, n_keypoints * 3, 1) # (x, y, vis) * 68 [cite: 264]
        )

    def forward(self, x):
        # Feed-forward (Single Path)
        x = self.stem(x)      # Stem
        x = self.stage1(x)    # P3
        x = self.down1(x)     # Down
        x = self.stage2(x)    # P4
        x = self.down2(x)     # Down
        x = self.stage3(x)    # P5 (Deepest features)

        # Head
        det = self.det_head(x)
        kpt = self.kpt_head(x)

        return det, kpt

    def fuse(self):
        """Reparameterization for Inference"""
        print("Fusing model layers for inference...")
        for m in self.modules():
            if isinstance(m, RepConv):
                m.fuse()
        return self

print("✓ YOLOMT Model updated with RepC2f & Bottleneck (Ready for Training)")

✓ Model fixed: AutoPad applied to RepConv
✓ YOLOMT Model updated with RepC2f & Bottleneck (Ready for Training)


In [ ]:
# ==========================================
# PART 3: Dataset Implementation (수정버전)
# ==========================================

import torch
from torch.utils.data import Dataset
import cv2
import numpy as np
import os
import glob
import scipy.io

class FaceLandmarkDataset(Dataset):
    def __init__(self, data_root, img_size=320, augment=False):
        self.data_root = data_root
        self.img_size = img_size
        self.augment = augment

        self.images = []

        # 데이터 경로 확인 및 리스트 생성
        if os.path.exists(data_root):
            # 300W-LP 폴더 구조에 맞춰 탐색
            # (재귀적으로 모든 하위 폴더의 jpg를 찾습니다)
            self.images = glob.glob(os.path.join(data_root, '**', '*.jpg'), recursive=True)

            # 더미/백업 파일 등이 섞일 수 있으므로 정렬
            self.images = sorted(self.images)

            print(f"✓ Found {len(self.images)} images in {data_root}")
        else:
            raise FileNotFoundError(f"❌ 데이터셋 경로가 없습니다: {data_root}")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # 1. 이미지 로드
        img_path = self.images[idx]
        img = cv2.imread(img_path)

        # 이미지 로드 실패 체크 (매우 중요!)
        if img is None:
            raise ValueError(f"❌ 이미지를 읽을 수 없습니다. 경로를 확인하세요: {img_path}")

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # 2. 라벨(.mat) 로드
        mat_path = img_path.replace('.jpg', '.mat')

        if not os.path.exists(mat_path):
            raise FileNotFoundError(f"❌ 라벨 파일(.mat)이 없습니다: {mat_path}")

        try:
            mat_data = scipy.io.loadmat(mat_path)
        except Exception as e:
            raise ValueError(f"❌ .mat 파일 손상됨: {mat_path} / 에러: {e}")

        # 3. 데이터 파싱
        try:
            pt2d = mat_data['pt2d'] # (2, 68)
            landmarks = torch.from_numpy(pt2d.T).float() # (68, 2)

            # Bounding box 계산
            x_min, y_min = landmarks.min(dim=0)[0]
            x_max, y_max = landmarks.max(dim=0)[0]
            w, h = x_max - x_min, y_max - y_min
            cx, cy = (x_min + x_max) / 2, (y_min + y_max) / 2

            # 정규화 (0-1)
            img_h, img_w = img.shape[:2]

            # ⚠️ Zero Division 방지
            if img_w == 0 or img_h == 0:
                 raise ValueError(f"이미지 크기가 0입니다: {img_path}")

            bbox = torch.tensor([[cx/img_w, cy/img_h, w/img_w, h/img_h]])
            landmarks[:, 0] /= img_w
            landmarks[:, 1] /= img_h

            # 값 범위 체크 (0~1 벗어나는지 확인)
            # 300W-LP는 가끔 랜드마크가 이미지 밖으로 나가는 경우가 있음 -> 클램핑
            landmarks = torch.clamp(landmarks, 0.0, 1.0)

            target = {
                'bbox': bbox,
                'landmarks': landmarks,
                'visibility': torch.ones(68)
            }

        except Exception as e:
            raise RuntimeError(f"❌ 데이터 처리 중 수식 에러 발생 ({img_path}): {e}")

        # 4. 리사이즈 (Letterbox)
        img_resized = self.letterbox_resize(img, self.img_size)
        img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0

        return img_tensor, target

    def letterbox_resize(self, img, new_size):
        h, w = img.shape[:2]
        if h == 0 or w == 0: return img

        scale = min(new_size / h, new_size / w)
        new_h, new_w = int(h * scale), int(w * scale)

        img_resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

        pad_h = (new_size - new_h) // 2
        pad_w = (new_size - new_w) // 2

        img_padded = np.full((new_size, new_size, 3), 114, dtype=np.uint8)
        img_padded[pad_h:pad_h+new_h, pad_w:pad_w+new_w] = img_resized

        return img_padded

print("✅ 수정된 데이터셋 클래스 정의 완료 (에러 검출 모드)")

✅ 수정된 데이터셋 클래스 정의 완료 (에러 검출 모드)


In [ ]:

print("\n" + "="*70)
print("PART 4: Training Setup")
print("="*70)

# 하이퍼파라미터
EPOCHS = 5
BATCH_SIZE = 16
IMG_SIZE = 320
LR = 0.01
NUM_WORKERS = 2

print(f"Hyperparameters:")
print(f"  - Epochs: {EPOCHS}")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Image size: {IMG_SIZE}")
print(f"  - Learning rate: {LR}")

# Dataset & DataLoader
train_dataset = FaceLandmarkDataset(
    data_root=DATA_ROOT,
    img_size=IMG_SIZE,
    augment=True
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

print(f"\n✓ Dataset: {len(train_dataset)} samples")
print(f"✓ DataLoader: {len(train_loader)} batches")

# 모델
model = SimplifiedYOLOMT(nc=1, n_keypoints=68).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"✓ Model: {n_params/1e6:.2f}M parameters")

# Optimizer & Scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=0.0001)

print("✓ Optimizer: SGD")
print("✓ Scheduler: CosineAnnealingLR")


PART 4: Training Setup
Hyperparameters:
  - Epochs: 5
  - Batch size: 16
  - Image size: 320
  - Learning rate: 0.01
✓ Found 122450 images in /content/dataset/300W_LP

✓ Dataset: 122450 samples
✓ DataLoader: 7654 batches
✓ Model: 1.80M parameters
✓ Optimizer: SGD
✓ Scheduler: CosineAnnealingLR


In [ ]:
import math
import torch
import torch.nn as nn

# ==========================================
# 1. WingLoss (TypeError 해결 + 수식 보정)
# ==========================================
class WingLoss(nn.Module):
    def __init__(self, w=10.0, epsilon=2.0):
        super(WingLoss, self).__init__()
        self.w = w
        self.epsilon = epsilon
        # 공식: C = w - w * ln(1 + w/epsilon)
        self.C = self.w - self.w * math.log(1 + self.w / self.epsilon)

    def forward(self, pred, target):
        diff = torch.abs(pred - target)

        # Case 1: 오차가 작을 때 (diff < w) -> Log 함수로 민감하게 반응
        # Case 2: 오차가 클 때 (diff >= w) -> Linear 함수 (diff - C)
        loss = torch.where(diff < self.w,
                           self.w * torch.log(1 + diff / self.epsilon),
                           diff - self.C)
        return torch.mean(loss)

# ==========================================
# 2. YOLOMTLoss (이전과 동일, WingLoss 연결)
# ==========================================
class YOLOMTLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
        self.mse = nn.MSELoss(reduction='none')
        self.wing = WingLoss()  # 위에서 수정한 WingLoss 사용

        self.lambda_obj = 1.0
        self.lambda_box = 5.0
        self.lambda_kpt = 1.0

    def make_grid_targets(self, pred_det, targets):
        B, C, H, W = pred_det.shape
        device = pred_det.device

        target_obj = torch.zeros((B, 1, H, W), device=device)
        target_box = torch.zeros((B, 4, H, W), device=device)
        target_kpt = torch.zeros((B, 136, H, W), device=device)

        mask_indices = []
        gt_bboxes = targets['bbox']
        gt_kpts = targets['landmarks']

        for b in range(B):
            box = gt_bboxes[b]
            kpt = gt_kpts[b]

            gx = int(box[0] * W)
            gy = int(box[1] * H)
            gx = max(0, min(gx, W - 1))
            gy = max(0, min(gy, H - 1))

            target_obj[b, 0, gy, gx] = 1.0
            target_box[b, :, gy, gx] = box
            target_kpt[b, :, gy, gx] = kpt.view(-1)
            mask_indices.append((b, gy, gx))

        return target_obj, target_box, target_kpt, mask_indices

    def forward(self, pred_det, pred_kpt, targets):
        target_obj, target_box, target_kpt, mask_indices = self.make_grid_targets(pred_det, targets)

        pred_conf = pred_det[:, 4:5, :, :]
        loss_obj = self.bce(pred_conf, target_obj).mean()

        if len(mask_indices) > 0:
            pos_mask = target_obj.bool()

            # --- Box Loss ---
            pred_box_vec = pred_det[:, :4, :, :].permute(0, 2, 3, 1)[pos_mask.squeeze(1)]
            target_box_vec = target_box.permute(0, 2, 3, 1)[pos_mask.squeeze(1)]
            loss_box = self.mse(torch.sigmoid(pred_box_vec), target_box_vec).mean()

            # --- Keypoint Loss ---
            pred_kpt_vec = pred_kpt.permute(0, 2, 3, 1)[pos_mask.squeeze(1)]
            pred_kpt_vec = torch.sigmoid(pred_kpt_vec)

            # (N, 204) -> (N, 136) 변환 (x,y 좌표만 추출)
            pred_kpt_vec = pred_kpt_vec.view(-1, 68, 3)[:, :, :2].reshape(-1, 136)

            target_kpt_vec = target_kpt.permute(0, 2, 3, 1)[pos_mask.squeeze(1)]
            loss_kpt = self.wing(pred_kpt_vec, target_kpt_vec)

        else:
            loss_box = torch.tensor(0.0, device=pred_det.device)
            loss_kpt = torch.tensor(0.0, device=pred_det.device)

        total_loss = (self.lambda_obj * loss_obj) + \
                     (self.lambda_box * loss_box) + \
                     (self.lambda_kpt * loss_kpt)

        return total_loss, (loss_obj.item(), loss_box.item(), loss_kpt.item())

In [ ]:
# ==========================================
# PART 5: Training (YOLO Loss 적용 버전)
# ==========================================

# 1. Loss 교체
criterion = YOLOMTLoss().to(device)

# 2. Optimizer 초기화 (새 모델 파라미터로 다시 연결 필수!)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.937, weight_decay=0.0005)

model.train()

print("🚀 YOLO Training Started...")

for epoch in range(EPOCHS):
    epoch_loss = 0.0

    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}')

    for batch_idx, (images, targets) in enumerate(pbar):
        images = images.to(device)

        # Targets 딕셔너리 값들도 GPU로 이동
        targets['bbox'] = targets['bbox'].to(device).float().squeeze(1) # [B, 1, 4] -> [B, 4] 주의!
        targets['landmarks'] = targets['landmarks'].to(device).float()

        # Forward (모델 구조 수정 후 det, kpt는 [B, C, H, W] 형태임)
        det_out, kpt_out = model(images)

        # Loss 계산 (이제 차원을 맞출 필요 없이 바로 넣습니다)
        loss, loss_items = criterion(det_out, kpt_out, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
        optimizer.step()

        epoch_loss += loss.item()

        pbar.set_postfix({
            'Total': f'{loss.item():.4f}',
            'Obj': f'{loss_items[0]:.4f}',
            'Box': f'{loss_items[1]:.4f}',
            'Kpt': f'{loss_items[2]:.4f}'
        })

🚀 YOLO Training Started...


Epoch 5/5: 100%|██████████| 7654/7654 [12:30<00:00, 10.20it/s, Total=0.0509, Obj=0.0013, Box=0.0001, Kpt=0.0490]


In [ ]:
def compare_prediction(model, dataset, idx=0, conf_thresh=0.5):
    model.eval()

    # --- 데이터 가져오기 ---
    img_tensor, target = dataset[idx]

    # 이미지 변환
    img_np = img_tensor.permute(1, 2, 0).cpu().numpy().copy()
    img_np = (img_np * 255).astype(np.uint8)
    img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    h, w = img_np.shape[:2]

    # --- 모델 추론 ---
    input_tensor = img_tensor.unsqueeze(0).to(device)
    with torch.no_grad():
        det_out, kpt_out = model(input_tensor)

    # --- 예측값 디코딩 ---
    pred_conf = torch.sigmoid(det_out[0, 4, :, :])
    gy, gx = np.unravel_index(torch.argmax(pred_conf).cpu(), pred_conf.shape)
    score = pred_conf[gy, gx].item()

    print(f"[{idx}번 샘플] 감지 확률: {score*100:.2f}% (Grid: {gx}, {gy})")

    if score < conf_thresh:
        print("⚠️ 감지 확률이 낮아 시각화를 건너뜁니다.")
        return

    # 1) 예측 랜드마크 (Red)
    kpt_vec = kpt_out[0, :, gy, gx]
    kpt_vec = torch.sigmoid(kpt_vec).cpu().numpy()

    pred_x = kpt_vec[0::3] * w
    pred_y = kpt_vec[1::3] * h

    # 2) 정답 랜드마크 (Green)
    gt_kpts = target['landmarks']
    gt_x = gt_kpts[:, 0].numpy() * w
    gt_y = gt_kpts[:, 1].numpy() * h

    # --- 그리기 ---
    vis_img = img_np.copy()

    # 정답 그리기 (초록색)
    for px, py in zip(gt_x, gt_y):
        cv2.circle(vis_img, (int(px), int(py)), 3, (0, 255, 0), -1)

    # 예측 그리기 (빨간색)
    for px, py in zip(pred_x, pred_y):
        cv2.circle(vis_img, (int(px), int(py)), 2, (0, 0, 255), -1)

    # 3) 정답 박스 (초록색) - 🔧 [수정된 부분]
    gt_bbox = target['bbox']

    # .flatten()을 추가하여 (1, 4) -> (4,)로 펴줍니다.
    bx, by, bw, bh = gt_bbox.flatten().numpy()

    x1 = int((bx - bw/2) * w)
    y1 = int((by - bh/2) * h)
    x2 = int((bx + bw/2) * w)
    y2 = int((by + bh/2) * h)
    cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # 결과 출력
    plt.figure(figsize=(12, 12))
    plt.imshow(cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB))
    plt.title(f"Green: GT / Red: Pred (Conf: {score:.2f})")
    plt.axis('off')
    plt.show()

# 다시 테스트 실행
print("🔍 0번 이미지 테스트")
compare_prediction(model, train_dataset, idx=29)

In [ ]:
# ==========================================
# 학습 종료 후 모델 저장 및 다운로드
# ==========================================
import torch

# 1. 파일 이름 정의
save_path = 'yolomt_final2.pt'

# 2. 모델 상태 저장 (가중치 + 옵티마이저 정보 포함)
torch.save({
    'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': epoch_loss,
}, save_path)

print(f"✅ 모델이 '{save_path}'로 저장되었습니다.")

# 3. 내 컴퓨터로 다운로드 (Google Colab 전용)
try:
    from google.colab import files
    files.download(save_path)
    print("⬇️ 다운로드가 시작됩니다...")
except ImportError:
    print("⚠️ Google Colab 환경이 아닙니다. 파일이 로컬 경로에 저장되었습니다.")

✅ 모델이 'yolomt_final2.pt'로 저장되었습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ 다운로드가 시작됩니다...


In [ ]:
import torch
import onnx
from onnxsim import simplify

# 1. 설정
IMG_SIZE = 320
BATCH_SIZE = 1
onnx_path = 'yolomt_best2.onnx'
# 저장된 pt 파일 경로 확인!
ckpt_path = '/content/drive/yolomt_best2.pt'

# 2. 모델 재성성 및 로드 (수정된 클래스 적용을 위해 필수)
print("♻️ Re-initializing model with fixed RepConv...")
model = SimplifiedYOLOMT(nc=1, n_keypoints=68).to(device)

if os.path.exists(ckpt_path):
    print(f"🔄 Loading weights from {ckpt_path}...")
    checkpoint = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
else:
    print("⚠️ Checkpoint file not found! Using random weights for test.")

model.eval()

# 3. Fuse (이제 에러가 안 날 것입니다)
print("🔧 Fusing RepConv layers...")
model.fuse()

# 4. Export
dummy_input = torch.randn(BATCH_SIZE, 3, IMG_SIZE, IMG_SIZE).to(device)
print(f"🚀 Exporting to ONNX...")

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    verbose=False,
    opset_version=11,
    input_names=['input'],
    output_names=['det_out', 'kpt_out'],
    do_constant_folding=True
)
print(f"✅ ONNX Exported: {onnx_path}")

# 5. Simplify
try:
    model_simp, check = simplify(onnx_path)
    onnx.save(model_simp, onnx_path)
    print("✅ ONNX Simplified!")
except Exception as e:
    print(f"⚠️ Simplify failed: {e}")

# 6. Download
from google.colab import files
files.download(onnx_path)

♻️ Re-initializing model with fixed RepConv...
⚠️ Checkpoint file not found! Using random weights for test.
🔧 Fusing RepConv layers...
Fusing model layers for inference...
🚀 Exporting to ONNX...
✅ ONNX Exported: yolomt_best2.onnx
✅ ONNX Simplified!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>